In [23]:
import pandas as pd

# Read the CSV dataset
file_path = '../datasets/dataset_latest.csv'
df = pd.read_csv(file_path).head(5) # Add .head(100) if you want to limit the number of rows

df = df.fillna('')

print(str(len(df)) + ' rows')
df.head()

5 rows


,id,title,summary,year,certificate,runtime,runtime_mins,rating,votes,director_1,...,writer_2,writer_3,cast_1,cast_2,cast_3,genre_1,genre_2,genre_3,synopsis,img
0,15239678,Dune: Part Two,Paul Atreides unites with Chani and the Fremen...,2024,PG-13,2h 46m,166,8.7,360000,Denis Villeneuve,...,Jon Spaihts,Frank Herbert,Timothée Chalamet,Zendaya,Rebecca Ferguson,Action,Adventure,Drama,Following the destruction of House Atreides by...,https://m.media-amazon.com/images/M/MV5BN2QyZG...
1,14539740,Godzilla x Kong: The New Empire,"Two ancient titans, Godzilla and Kong, clash i...",2024,PG-13,1h 55m,115,6.5,37000,Adam Wingard,...,Simon Barrett,Jeremy Slater,Rebecca Hall,Brian Tyree Henry,Dan Stevens,Action,Adventure,Fantasy,"Three years after defeating Mechagodzilla, Kon...",https://m.media-amazon.com/images/M/MV5BY2QwOG...
2,23137904,Rebel Moon - Part Two: The Scargiver,Kora and surviving warriors prepare to defend ...,2024,PG-13,2h 2m,122,5.2,17000,Zack Snyder,...,Kurt Johnstad,Shay Hatten,Sofia Boutella,Djimon Hounsou,Ed Skrein,Action,Adventure,Drama,Kora and surviving warriors prepare to defend ...,https://m.media-amazon.com/images/M/MV5BYmQ2OD...
3,21692408,Kung Fu Panda 4,After Po is tapped to become the Spiritual Lea...,2024,PG,1h 34m,94,6.4,30000,Mike Mitchell,...,Glenn Berger,Darren Lemke,Jack Black,Awkwafina,Viola Davis,Animation,Action,Adventure,"On top of a mountain, a bunch of mountain goat...",https://m.media-amazon.com/images/M/MV5BZDY0Yz...
4,1160419,Dune,A noble family becomes embroiled in a war for ...,2021,PG-13,2h 35m,155,8.0,851000,Denis Villeneuve,...,Denis Villeneuve,Eric Roth,Timothée Chalamet,Rebecca Ferguson,Zendaya,Action,Adventure,Drama,The story opens with a woman telling a portion...,https://m.media-amazon.com/images/M/MV5BMDQ0Nj...


In [24]:
from dotenv import load_dotenv, find_dotenv
import json
import os
from openai import OpenAI
import openai
from groq import Groq

load_dotenv(find_dotenv())

# client = OpenAI(
#     api_key=os.getenv('OPENAI_API_KEY'),
# )

client = Groq(
    api_key=os.getenv('GROQ_API_KEY'),
)

# llm_model_name = "llama3-8b-8192"
llm_model_name="mixtral-8x7b-32768"
# llm_model_name="gpt-3.5-turbo-0125"

def analyze_sentiment(title, synopsis):

    prompt_template = f"""
        For the following movie title '{title}' and its synopsis '{synopsis}', analyze the synopsis of a movie and assign a sentiment score ranging from 0 to 10. (0 being extremely sad/negative, 1 being very sad, 2 being sad/negative, 3 being somewhat sad, 4 being sad closer to neutral, 5 being neutral, 6 being happy/positive closer to neutral, 7 being somewhat happy/positive, 8 being happy/positive, 9 being very happy, 10 being extremely happy/positive). Consider the emotional tone of the synopsis, especially how the movie concludes. If the movie has a bittersweet or mature ending, take into account themes of sacrifice, loss, or unresolved conflicts, and assign a lower score. Additionally, if the movie tackles controversial themes such as slavery, abortion, or war, consider lowering the score accordingly.
        Only provide a sentiment score along with a ***specific*** justification/reasoning as to why the movie received with the score and recommended audience in about 30 words. Output needs to be in a JSON format.
        
        Example JSON Output:
        {{
        	"Sentiment Score": 3,
        	"Justification": "This film depicts a poignant love story set against the backdrop of societal expectations and restrictions. The themes of love, sacrifice, and longing contribute to a somewhat sad and bittersweet overall tone.",
        	"Recommended Audience": "This film may resonate with viewers who enjoy slow-burn romance narratives and are open to exploring themes of societal constraints and the complexities of human relationships."
        }}
    """

    messages = [{
            "role": "system",
            "content": "Please generate output in JSON format exclusively, avoiding any additional text or explanations.",
        },
        {
            "role": "user",
            "content": prompt_template
        }
    ]

    stream = client.chat.completions.create(
        model=llm_model_name,
        messages=messages,
        max_tokens=200,
        temperature=0.5,
        frequency_penalty=0,
        presence_penalty=0,
        response_format={ "type": "json_object" }
    )
    return json.loads(stream.choices[0].message.content)

In [5]:
story = "On top of a mountain, a bunch of mountain goats are mining, when Tai Lung shows up and destroys the mountain. Meanwhile, Po is fighting off an evil stingray and once he's done, Shifu asks him to talk, but Po just wants to hang around and have fun with the animals of the valley. Shifu finally gets Po to talk and on the steps to the Jade Palace, he tells Po, who is holding Oogways staff, to find a successor as the dragon warrior, much to Pos annoyance. The following day, Po and Shifu watch all 5 candidates perform, but Po struggles to pick one and leaves it to his finger to decide, of which eventually chooses him, much to Shifus anger. Po walks back to the palace to find someone sneaking in and attempting to steal items. Po and the fox named 'Zhen' battle each other and Po defeats her, sending her to prison. Later, Po hears of Tai Lungs return and goes to visit Zhen, who gave him a clue earlier. In the prison, Zhen tells him that Tai Lung isn't back, but is instead, the work of an evil sorceress shape shifter called the Chameleon. Po wants to take her down alone, but Zhen says if he is going to defeat her, he'll need her help. Po and Zhen head off to juniper city, to the village's excitement, however both of Pos dads, Ping and Li worry about him and soon go after him. Po and Zhen enter a restaurant and destroy the place, as Zhen tries swindling them out of mahjong tiles, but the duo quickly escape with the captain, who steers the boat, but also drinks a lot. On the boat, Po has a vision of what the Chameleon wants and the duo arrive at Juniper City. In the streets, Po learns that Zhen is a wanted thief and the police go after them, but they manage to escape to the Den of Thieves, where Zhen reconciles with Han, who took her in when she was younger. The next day, Po and Zhen escape to the lair of the Chameleon and go in. The Chameleon captures Po in a cage and Zhen steals the staff and gives it to the Chameleon, revealing she has been working for her the entire time. The Chameleon tells him that her true passion is Kung Fu, but was turned down by every training place, because she was too small and decided on sorcery instead. Po tries to escape, but cannot break the cage, so he destroys the ground and falls into a pit of guards. However, Po comes back and asks Zhen to give him back the staff, but it is the Chameleon in disguise and she chucks him off the cliff. The Chameleon uses the staff to gather all the power from all the Kung Fu masters of the past. Zhen realizes what she had done and goes to find Po. Po talks to his dads about change and they tell him that change isn't always a bad thing. Po becomes encouraged by this and goes back to stop the Chameleon, but Zhen doesn't want him to get killed and tries to stop him, but fails. After this, Zhen goes back to the Den of Thieves and encourages them to help her defeat the Chameleon and save Pos life. Back in the room of the lair, Po tells Tai Lung that he lost the staff of Wisdom, which makes him lose more faith in Po. The Chameleon and Po fight each other, but the Chameleon uses the power of the masters to give her more strength, before she transforms into all the masters in one, which greatly amazes Po. Zhen tries to get involved, but ends up falling with the rubble. The Chameleon transforms into Po and the true Po slaps the imaginary ones butt. The Chameleon captures him in yet another cage, but Zhen uses the move Po had taught her earlier to defeat the Chameleon and give the Kung Fu villains their powers. Po uses the staff of wisdom to send all the villains back through the door, into the spirit realm. Back at the jade palace, Po tells Shifu that he has chosen Zhen as his successor, which makes Shifu spit out his tea and he goes to the tree to meditate all of his stresses away. Zhen doesn't believe she can be, but Po gets the furious five to help train her."

result = analyze_sentiment('Kung Fu Panda 4', story)

print(result)

{'Sentiment Score': 7, 'Justification': 'This film features a thrilling and humorous Kung Fu adventure with themes of loyalty, self-discovery, and redemption, resulting in a positive and uplifting overall tone.', 'Recommended Audience': 'Recommended for viewers of all ages who enjoy action-packed animations with a touch of humor and a strong message of friendship and teamwork.'}


In [25]:
from tqdm import tqdm

# Create empty columns
df['sentiment_score'] = ''
df['sentiment_reason'] = ''
df['recommended_audience'] = ''

# Function to apply sentiment analysis and update DataFrame
def apply_sentiment_analysis(row):
    try:
        result = analyze_sentiment(row['title'], row['synopsis'])
        row['sentiment_score'] = result.get('Sentiment Score', '')
        row['sentiment_reason'] = result.get('Justification', '')
        row['recommended_audience'] = result.get('Recommended Audience', '')
    except Exception as e:
        print(f"Error processing row: {e}")
        row['sentiment_score'] = ''
        row['sentiment_reason'] = ''
        row['recommended_audience'] = ''
    return row

# Apply sentiment analysis function to each row with tqdm progress bar
tqdm.pandas()
df = df.progress_apply(apply_sentiment_analysis, axis=1)

df.head()

100%|███████████| 5/5 [00:53<00:00, 10.63s/it]


,id,title,summary,year,certificate,runtime,runtime_mins,rating,votes,director_1,...,cast_2,cast_3,genre_1,genre_2,genre_3,synopsis,img,sentiment_score,sentiment_reason,recommended_audience
0,15239678,Dune: Part Two,Paul Atreides unites with Chani and the Fremen...,2024,PG-13,2h 46m,166,8.7,360000,Denis Villeneuve,...,Zendaya,Rebecca Ferguson,Action,Adventure,Drama,Following the destruction of House Atreides by...,https://m.media-amazon.com/images/M/MV5BN2QyZG...,6,"This movie features a mix of action, adventure...",This film is recommended for viewers who enjoy...
1,14539740,Godzilla x Kong: The New Empire,"Two ancient titans, Godzilla and Kong, clash i...",2024,PG-13,1h 55m,115,6.5,37000,Adam Wingard,...,Brian Tyree Henry,Dan Stevens,Action,Adventure,Fantasy,"Three years after defeating Mechagodzilla, Kon...",https://m.media-amazon.com/images/M/MV5BY2QwOG...,7,The movie has a triumphant and exhilarating to...,This film is recommended for viewers who enjoy...
2,23137904,Rebel Moon - Part Two: The Scargiver,Kora and surviving warriors prepare to defend ...,2024,PG-13,2h 2m,122,5.2,17000,Zack Snyder,...,Djimon Hounsou,Ed Skrein,Action,Adventure,Drama,Kora and surviving warriors prepare to defend ...,https://m.media-amazon.com/images/M/MV5BYmQ2OD...,7,The synopsis suggests a thrilling and action-p...,This film is recommended for viewers who enjoy...
3,21692408,Kung Fu Panda 4,After Po is tapped to become the Spiritual Lea...,2024,PG,1h 34m,94,6.4,30000,Mike Mitchell,...,Awkwafina,Viola Davis,Animation,Action,Adventure,"On top of a mountain, a bunch of mountain goat...",https://m.media-amazon.com/images/M/MV5BZDY0Yz...,7,The movie has a positive and uplifting tone wi...,This animated film is recommended for viewers ...
4,1160419,Dune,A noble family becomes embroiled in a war for ...,2021,PG-13,2h 35m,155,8.0,851000,Denis Villeneuve,...,Rebecca Ferguson,Zendaya,Action,Adventure,Drama,The story opens with a woman telling a portion...,https://m.media-amazon.com/images/M/MV5BMDQ0Nj...,4,Dune presents a complex and nuanced narrative ...,This film may appeal to those who enjoy intric...


In [27]:
file_path = 'filmdataset_latest.csv'
df.to_csv(file_path, index=False)